# General framework

In the regression problem of supervised learning, a machine learning model with parameter $\theta$ is determined by minimizing an objective function that has the form of a sum over individual data unit:
$$
Q(\theta)=\frac{1}{N}\sum_{i=1}^{N} l(f(x_i|\theta),y_i),
$$
where $\hat{y}_i\equiv f(x_i|\theta)$ is the prediction given by the model.
The right hand side of the equation can be interpreted as the expansion of the total objective function.

The online learning problem can be framed as how the model parameter $\theta$ updates with respect to the addition of more expansion component or incoming data. Suppose the optimal solution of the objective function $\theta_{0}$ is known with respect to the original dataset $\{x,y\}_{0}$ of size $N_0$:

$$
\theta_{0}=\arg \min_{\theta} \frac{1}{N_0}\sum_{i=1}^{N_0} l(f(x_i|\theta),y_i).
$$
Then given another data set $\{x,y\}_{1}$ with sieze $N_1$, what is the optimal solution  $\theta$ with respect to the total objective function.

$$
Q(\theta)=\frac{1}{N_0+N_1}\sum_{i=1}^{N_0+N_1} l(f(x_i|\theta),y_i).
$$

This question may be attacked from two angles:
* starting from $\theta_0$, find, e.g., the iterative expression of the total optimul solution $\theta_{tot}$ in terms of $\theta_0$ and $\{x,y\}_{1}$.
* solve respectively the optimal solution of each data set and find a way to combine them

The first angle is related to ,e.g., the online recommendation system. The second angle is related to the distributed training by assigning the same model to different dataset.

## Special case 1

If in regardless of the data set size, the objective function always belong to a  class of analytic functions $f$ characterized by some finite number of parameters $\{w \}$, then Eq. (1) can be rewritten as
$$
Q(\theta)=f(\theta|w(\{x,y\})).
$$
It is then possible to achieve a general analytic expression of the optimul solution as a function of parameter $w$:
$$
\theta=p(w(\{x,y\}))
$$

Next, we demonstrate two examples. The algorithm discussion can be find [here](OML_algorithm.ipynb)

## Example 1: output average 

In this simplest case, the model return a single value $\omega$ as the prediction. The loss function for the model is
$$
\frac{1}{N}\sum (\omega-Y_i)^2+\lambda \omega^2, 
$$
where $\gamma$ is a regularization term that gives a punishment for dividing a parent node thus reduce the complexity of decision tree structure. Although simple, it is the model used at each tree node in the **XGboost**.

In this case, the parameter of the model is $\omega$ which is the same as the prediction and we donnot need state variables to assistants the calculate the online updating of $\omega$ since the minimization of the loss function gives:
\begin{align}
\omega=&\frac{\bar{Y}}{1+\lambda},\\
loss=&\overline{Y^2}-(1+\lambda)\omega^2.
\end{align}

Thus we can easily get a online updating expression for both of them as
\begin{align}
i\omega_i=&(i-1)\omega_{i-1}+\frac{Y_i}{1+\lambda},\\
i[loss_i+(1+\lambda)\omega_i^2]=&(i-1)[loss_{i-1}+(1+\lambda)\omega_{i-1}^2]+Y_i^2.
\end{align}

We can also get the batch updating expression for both of them as
\begin{align}
i\omega_i=&j\omega_{j}+\sum_{j}^{i}\frac{Y}{1+\lambda},\\
i[loss_i+(1+\lambda)\omega_i^2]=&j(loss_{j}+(1+\lambda)\omega_{j}^2)+\sum_{j}^{i}Y^2.
\end{align}

### Example 2: linear regression

The model of linear regression is given by $\hat{y}=\boldsymbol{\theta}\cdot \boldsymbol{x}$ and the objective function can be given by the mean square error as:
$$
MSE=\frac{1}{N}\sum_{i}(\theta_{j} x_{i}^{j}-y_i)^2.
$$
We can see that in terms of the model parameter $\boldsymbol{\theta}$, the MSE is always a second order polynomial function with the form:
$$
MSE=\frac{1}{N}[a^{mn}\theta_{m}\theta_{n}-2 b^{k}\theta_{k}+c],
$$
with the parameters $a^{mn},b^{k},c$ given by
\begin{align}
a^{mn}=& \sum_i x_{i}^{m} x_{i}^{n},\\
b^{k}=&\sum_i x_{i}^{k} y_{i},\\
c=&\sum_i y_{i} y_{i}.
\end{align}

Normal Equation

The analytic result for the least MSE reads:
\begin{align} 
\hat{\theta}=\boldsymbol{a}^{-1}\boldsymbol{b}.
\end{align}

And the minimal of the MSE is given by
\begin{align}
\frac{1}{N}(\hat{\boldsymbol{\theta}}^{T} \cdot \boldsymbol{a} \cdot \hat{\boldsymbol{\theta}}-2\boldsymbol{b}^T \cdot \hat{\boldsymbol{\theta}} +c) =& \frac{1}{N}[(\boldsymbol{a}^{-1}\boldsymbol{b})^{T}\boldsymbol{a}(\boldsymbol{a}^{-1}\boldsymbol{b})-2\boldsymbol{b}^T(\boldsymbol{a}^{-1}\boldsymbol{b})+c] \\
=&\frac{1}{N}(-\boldsymbol{b}^T\boldsymbol{a}^{-1}\boldsymbol{b}+c)
\end{align}

#### Computational Complexity
##### **Normal Equation**
If we use normal equation to compute the optimal solution, calculating the matrix $xx^{T}$ needs $O(nd^{2})$ with $n$ the sample size and $d$ the feature dimension.  The computational complexity of inverting the matrix is typically about $O(d^{2.4})$ to $O(d^{3})$ (depending on the implementation). So it takes $O(nd^{2}+d^{3})$ to get the inverse matrix. 

Then the calculation of $\hat{\theta}$ takes $O(d^2)$ and MSE takes $O(d^2+d)$.

##### **RLS**
However, in the case of online training, if the data comes in sequence or cannot fit into the memory at once, we are forced to study the relation between $a_i^{-1}$ and $a_j^{-1}$ with $j>i$. Denoting $a_i^{-1}$ as $\boldsymbol{\Gamma}_{i}$The recursive least squares (RLS) algorithm gives a relation between the inverse matrices with respect two data set differ by one data point:
\begin{align}
\boldsymbol{\Gamma}_{i}=&\boldsymbol{\Gamma}_{i-1}-\frac{\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}
\end{align}
Each iteration takes $O(d^2)$ and with a data size $n$ it takes $O(nd^2)$. In contrary, if normal equation is used to calculate the optimal solution for each step, it will take $\sum_{i+1}^{n}O(id^2+d^3)=O(n^2d^2+nd^3)=O(nd^2\max(n,d))$. So the two methods differ by a factor $\max(n,d)$.

#### Derivation
The derivation relies on a  key identity:
\begin{align}
\boldsymbol{\Gamma}_{i-1}\boldsymbol{a}_{i-1}=\boldsymbol{\Gamma}_{i-1}(\boldsymbol{a}_{i}-\boldsymbol{x}_{i})=&1\\
\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}(\boldsymbol{a}_{i}-\boldsymbol{x}_{i})=&\boldsymbol{x}_{i}^{T}\\
\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{a}_{i}=&(1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i})\boldsymbol{x}_{i}^{T}\\
\frac{\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{a}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}=&\boldsymbol{x}_{i}^{T}\\
\frac{\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}=&\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i}.
\end{align}
It serves the purpose to switch from $\boldsymbol{\Gamma}_{i}$ to $\boldsymbol{\Gamma}_{i-1}$ when acting on $\boldsymbol{x}_{i}^{T}$.
Then for the iterative relation of $\boldsymbol{\Gamma}$:
\begin{align}
1=&\boldsymbol{\Gamma}_{i}(\boldsymbol{a}_{i-1}+\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T})\\1=&1+\Delta_{i}\boldsymbol{a}_{i-1}+\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\\0=&\Delta_{i}+\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\\\Delta_{i}=&-\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\\\boldsymbol{\Gamma}_{i}=&\boldsymbol{\Gamma}_{i-1}-\frac{\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}.
\end{align}
With RLS, we can derive the iterative expression for $\hat{\theta}$ and MSE 
\begin{align}
\hat{\boldsymbol{\theta}}_{i}=&\boldsymbol{\Gamma}_{i}\boldsymbol{b}_{i}=(\boldsymbol{\Gamma}_{i-1}-\frac{\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}})(\boldsymbol{b}_{i-1}+\boldsymbol{x}_{i}y_{i})\\=&\hat{\boldsymbol{\theta}}_{i-1}-\frac{\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}\boldsymbol{b}_{i-1}+\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}y_{i}\\=&\hat{\boldsymbol{\theta}}_{i-1}-\frac{\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}+\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}y_{i}\\=&\hat{\boldsymbol{\theta}}_{i-1}-\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})
\end{align}

\begin{align}
iM_{i}=&(-\boldsymbol{b}_{i}^{T}\boldsymbol{\Gamma}_{i}\boldsymbol{b}_{i}+c_{i})\\=&-(\boldsymbol{b}_{i-1}^{T}+\boldsymbol{x}_{i}^{T}y_{i})\hat{\boldsymbol{\theta}}_{i}+c_{i}\\=&-(\boldsymbol{b}_{i-1}^{T}+\boldsymbol{x}_{i}^{T}y_{i})(\hat{\boldsymbol{\theta}}_{i-1}-\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i}))+c_{i}\\=&-(\boldsymbol{b}_{i-1}^{T}\hat{\boldsymbol{\theta}}_{i-1}+\boldsymbol{x}_{i}^{T}y_{i}\hat{\boldsymbol{\theta}}_{i-1})+(\boldsymbol{b}_{i-1}^{T}+\boldsymbol{x}_{i}^{T}y_{i})\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})+c_{i}\\=&(i-1)M_{i-1}-y_{i}\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}+(\boldsymbol{b}_{i-1}^{T}+\boldsymbol{x}_{i}^{T}y_{i})\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})+y_{i}y_{i}\\=&(i-1)M_{i-1}-y_{i}\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}+\boldsymbol{b}_{i-1}^{T}\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})+\boldsymbol{x}_{i}^{T}y_{i}\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})+y_{i}y_{i}\\=&(i-1)M_{i-1}-y_{i}\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}+\boldsymbol{b}_{i-1}^{T}\frac{\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})+\boldsymbol{x}_{i}^{T}y_{i}\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})+y_{i}y_{i}\\=&(i-1)M_{i-1}-y_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})+(\frac{\boldsymbol{\theta}_{i-1}^{T}\boldsymbol{x}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}+y_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i})(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})\\=&(i-1)M_{i-1}+(\frac{\boldsymbol{\theta}_{i-1}^{T}\boldsymbol{x}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}+y_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}-y_{i})(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})\\=&(i-1)M_{i-1}+(\frac{\boldsymbol{\theta}_{i-1}^{T}\boldsymbol{x}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}+y_{i}\frac{\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}-y_{i})(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})\\=&(i-1)M_{i-1}+(\frac{\boldsymbol{\theta}_{i-1}^{T}\boldsymbol{x}_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}-\frac{y_{i}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}})(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})\\=&(i-1)M_{i-1}+\frac{(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})^{2}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}
\end{align}

The iterative relation of $\hat{\boldsymbol{\theta}}$ and MSE can also be derived as:
\begin{align}
\hat{\boldsymbol{\theta}}_{i}=&\hat{\boldsymbol{\theta}}_{i-1}-\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})\\
iM_{i}=&(i-1)M_{i-1}+\frac{(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})^{2}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}
\end{align}

We can see from both expressions that when the new added data follows exactly the prediciton by the model trained before, no update for either the model parameters or mean square error is needed.

It is also interesting to ask what the iterative relation is if data is removed instead of added. A good starting point is the relation 
\begin{align}
\frac{\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i}}{1-\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}}=&\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}.
\end{align}
Then we have the inverse iterative relations as: 
\begin{align}
\boldsymbol{\Gamma}_{i-1}=&\boldsymbol{\Gamma}_{i}+\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i},
\end{align}
\begin{align}
\hat{\boldsymbol{\theta}}_{i-1}=&\hat{\boldsymbol{\theta}}_{i}+\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i}-y_{i}),
\end{align}
\begin{align}
(i-1)M_{i-1}=&iM_{i}-\frac{(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i}-y_{i})^{2}}{1-\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}}.
\end{align}

\begin{align}
\boldsymbol{\Gamma}_{i}=&\boldsymbol{\Gamma}_{i-1}-\frac{\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}\\
\hat{\boldsymbol{\theta}}_{i}=&\hat{\boldsymbol{\theta}}_{i-1}-\boldsymbol{\Gamma}_{i}\boldsymbol{x}_{i}(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})\\
iM_{i}=&(i-1)M_{i-1}+\frac{(\boldsymbol{x}_{i}^{T}\hat{\boldsymbol{\theta}}_{i-1}-y_{i})^{2}}{1+\boldsymbol{x}_{i}^{T}\boldsymbol{\Gamma}_{i-1}\boldsymbol{x}_{i}}
\end{align}